Extract Transform Load (ETL) Lab

Objectives

After completing this lab you will be able to:

Read CSV and JSON file types.

Extract data from the above file types.

Transform data.

Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions



In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

Download Files


https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip


SET PATHS

In [2]:
tmp_file    = "temp.tmp"               # file used to store all extracted data
log_file    = "log_file.txt"            # all event logs will be stored in this file
target_file = "transformed_data.csv"   # file where transformed data is stored

EXTRACT

CSV Extract Function

In [3]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe 

JSON Extract Function

In [4]:
def extract_from_json(file_to_process): 
    dataframe = pd.read_json(file_to_process, lines=True) 
    return dataframe

XML Extract Function

In [5]:
def extract_from_xml(file_to_process): 
    dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        name = person.find("name").text 
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True) 
    return dataframe 

Extract Function

In [6]:
def extract(): 
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data 
     
    # process all csv files 
    for csvfile in glob.glob("*.csv"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True) 
         
    # process all json files 
    for jsonfile in glob.glob("*.json"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True) 
     
    # process all xml files 
    for xmlfile in glob.glob("*.xml"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True) 
         
    return extracted_data 

Transform
The transform function does the following tasks.

1. Convert height which is in inches to millimeter
2. Convert weight which is in pounds to kilograms

In [7]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

Loading

In [8]:
def load_data(target_file, transformed_data):
    transformed_data.to_csv(target_file)  

Logging

In [9]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("log_file.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

Running ETL Process


In [10]:
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract() 
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
 
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 
 
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
 
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
    name height weight  Unnamed: 0        Country  GDP_USD_billions Currency  \
0    NaN    NaN    NaN         0.0  United States          26854.60      NaN   
1    NaN    NaN    NaN         1.0          China          19373.59      NaN   
2    NaN    NaN    NaN         2.0          Japan           4409.74      NaN   
3    NaN    NaN    NaN         3.0        Germany           4308.85      NaN   
4    NaN    NaN    NaN         4.0          India           3736.88      NaN   
..   ...    ...    ...         ...            ...               ...      ...   
189  NaN    NaN    NaN       189.0          Nauru              0.15      NaN   
190  NaN    NaN    NaN       190.0         Tuvalu              0.06      NaN   
191  NaN    NaN    NaN         NaN            NaN               NaN      EUR   
192  NaN    NaN    NaN         NaN            NaN               NaN      GBP   
193  NaN    NaN    NaN         NaN            NaN               NaN      INR   

      Rate  
0      Na